In [1]:
import ee
import datetime
import os
import itertools
import sys

from pprint import pprint
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE

In [2]:
ee.Initialize()

In [3]:
# append upper folder into sys-path during run time so we can
# import our own moduls
sys.path.append('../Process_1_GEE_Python_Classification/')

# import the temporal_consistency_check class
from BackGround_modules.Class_5_Temporal_consistency_check import Temporal_consistency_check

In [4]:
# define basic parameters
year_range = [f'{i}_{i+2}' for i in range(1990,2020,3)]

In [5]:
year_range

['1990_1992',
 '1993_1995',
 '1996_1998',
 '1999_2001',
 '2002_2004',
 '2005_2007',
 '2008_2010',
 '2011_2013',
 '2014_2016',
 '2017_2019']

### Import Classified_random_img

In [6]:
# Define the threshold for sum_img of 10 classifications
Threshold_sum_classificatioin = 4

# import the classified imgs
Classified_Landsat_1990_2019  = [ee.ImageCollection(f"users/wangjinzhulala/North_China_Plain_Python/classification_img/Control_{year}")
                                  .sum().gte(Threshold_sum_classificatioin).set('name',year)   for year in year_range]

Classified_Sentinel_2014_2019 = [ee.ImageCollection(f"users/wangjinzhulala/North_China_Plain_Python/classification_img/Sentinel_Landsat_{year}")
                                  .sum().gte(Threshold_sum_classificatioin).set('name',year)   for year in year_range[-2:]]

In [7]:
Classified_imgs

NameError: name 'Classified_imgs' is not defined

In [17]:
# Use the temporal_flag to indicate weather to use Sentinel classified imgs

temporal_flag = 'Landsat' #'Landsat'/'Sentinel'

# decide which classified img to be used
if temporal_flag == 'Landsat':
    Classified_imgs = Classified_Landsat_1990_2019
    
elif temporal_flag == 'Sentinel':
    Classified_imgs = Classified_Landsat_1990_2019[:-2] + Classified_Sentinel_2014_2019

else:
    print('Incorrect temporal_flag!')

### Perform temporal consistency check

In [37]:
# get the checked results
Iteration_num = 5

Iter_temporal_check_instaces = Temporal_consistency_check(Classified_imgs,3,Iteration_num).Iterate_the_check()

Check length is    ---> 3
Check weights is   ---> [3, 1, 1]
Check threshold is ---> 4


In [56]:
# convert the Iter_temporal_check_instaces to a Dataframe
Iter_temporal_check_instaces_df = pd.DataFrame(Iter_temporal_check_instaces).T
Iter_temporal_check_instaces_df.columns = year_range
Iter_temporal_check_instaces_df['Iteration'] = range(1,Iteration_num+1)

# Reorder the columns
cols = Iter_temporal_check_instaces_df.columns
Iter_temporal_check_instaces_df = Iter_temporal_check_instaces_df[list(cols[-1:]) + list(cols[:-1])] 
Iter_temporal_check_instaces_df

,Iteration,1990_1992,1993_1995,1996_1998,1999_2001,2002_2004,2005_2007,2008_2010,2011_2013,2014_2016,2017_2019
0,1,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume..."
1,2,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume..."
2,3,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume..."
3,4,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume..."
4,5,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.Image({\n ""type"": ""Invocation"",\n ""argume..."


In [55]:
list(cols[-1:]) + list(cols[:-1])

['Iteration',
 '1990_1992',
 '1993_1995',
 '1996_1998',
 '1999_2001',
 '2002_2004',
 '2005_2007',
 '2008_2010',
 '2011_2013',
 '2014_2016',
 '2017_2019']

##### Compute the accuracy after temporal-check

In [28]:
# import the untouched control points, which are diveded by a random column with a 101 random state
Sample_Landsat  = ee.FeatureCollection("users/wangjinzhulala/North_China_Plain_Python/Sample_Points/Control_points")\
                    .randomColumn('split', 101)\
                    .filterMetadata('split','less_than',0.25)

Sample_Sentinel = ee.FeatureCollection("users/wangjinzhulala/North_China_Plain_Python/Sample_Points/Sentinel_2017_2019")\
                    .randomColumn('split', 101)\
                    .filterMetadata('split','less_than',0.25)

##### Add one of the results to map

In [22]:
Map = geemap.Map()
Map.setCenter(115.4508, 35.2492,10)

year_idx = 3

Map.add_basemap('HYBRID')
Map.addLayer(Classified_imgs[year_idx] ,{'min':0,'max':1},'origin')
Map.addLayer(Iter_temporal_check_instaces[1][year_idx] ,{'min':0,'max':1},'Iter_0')
Map.addLayer(Iter_temporal_check_instaces[2][year_idx] ,{'min':0,'max':1},'Iter_1')
Map.addLayer(Iter_temporal_check_instaces[3][year_idx] ,{'min':0,'max':1},'Iter_2')

Map

Map(center=[35.2492, 115.4508], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButt…